# Open Food Facts Course Project - Cleaning, manipulating and visualizing

First of all, let's import usefull libraries for the project and make matplolib displaying graphs inline the Notebook.

In [1]:
from os import path # For filepath manipulation
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn

Next step will be to load the Open Food Facts CSV file, which in fact is a TSV file (Cells are separated with tabs).
Loading this file will take a consequent amount of time, as the file is 1Gb big. First thing to do will be to perform some cleanup and removal of useless data, and save the result as a new CSV file.

This new CSV file will be used as datasource in this Notebook using the first **MAX_ENTRY_TO_LOAD** rows while coding.

**Don't forget to set MAX_ENTRY_TO_LOAD = None when coding is finished.** Otherwise, only a subset of the data will be processed.

* Note that this Notebook checks if the cleaned Datafile exists and create it otherwise. This process relies on three functions: First one will load the original TSV file, the second one will cleanup the orignal data and the third one will dump the cleand data into a new CSV file.

The following global constant can be adapted to suit your needs

In [2]:
# Filename of the original TSV file
ORIGINAL_TSV_FILENAME = path.join('data','OpenFoodFacts.tsv')

# Filename of the cleaned data build in this Notebook
CLEANED_CSV_FILENAME = path.join('data','OpenFoodFacts-cleaned.csv')

# If set to true, the original data file loading process is forced, event if the
# cleaned CSV file exists. Should be set to **True** when coding is finished
FORCE_LOAD_ORIGINAL_FILE = False # PRoduction value = True

# Maximum NaN percentage accepted in a column. If above, the column is dropped.
MAX_NAN_PERCENT_VALUE = 80

# Number of rows loaded from cleaned CSV file. Usefull while coding, this value should be
# set ton **None** when coding is finished.
MAX_ENTRY_TO_LOAD = 20000 # Production value = None

# List of columns that will be removed from the Dataset (useless one)
COLUMNS_TO_DROP = [
    'creator', 'brands', 'brands_tags', 'categories','main_category', 'countries',
    'countries_tags', 'additives', 'additives_tags', 'categories_tags', 'states',
    'states_en', 'states_tags', 'url', 'quantity', 'packaging_tags', 'packaging',
    'created_t', 'last_modified_t', 'pnns_groups_1', 'pnns_groups_2', 'image_url',
    'image_small_url', 'code'
]


## A. Importing and cleaning the data

Importing the datasource is done using Pandas **read_csv** method, using parameter **sep="\t"** as the content of the file is a tabulation spearated CSV file.

* Note that I've set the **low_memory** option to False in order to avoid warnings when loading the file. Number of columns is quite important and the process to determine the column dtype is too consuming*


### Some function definitions

#### Data loading function.

In [3]:
def loadOriginalTsvFile(filename):
    print("Loading data from file",ORIGINAL_TSV_FILENAME)
    print("Please wait...")
    df = pd.read_csv(ORIGINAL_TSV_FILENAME,sep="\t",low_memory=False)
    print("Loading process terminated.")
    return df


#### Function to dump the cleaned data into a new CSV file

In [4]:
def dumpCleanedCsvFile(df,filename):
    print("Dumping the cleaned Dataframe into file",CLEANED_CSV_FILENAME)
    print("Please wait...")
    df.to_csv(CLEANED_CSV_FILENAME)
    print("Dumping process terminated")

#### Cleanup function that process the Dataframe returned by the **loadOriginalTsvFile()** function

This function will perform cleanup actions on the whole dataset. Further in this Notebook, more cleaning actions will come while we discover the content of the Open Food Facts database.

Here is a list of the cleaning actions done here:

* Drop unused column defined in the global parameter COLUMNS_TO_DROP
* Drop columns where the percentage of null values is above MAX_NAN_PERCENT_VALUE
* Drop rows where **product_name** or **countries_en** column are empty
* Drop rows with duplicates in 'product_name' column

In [5]:
def cleanOriginalData(df):

    print("Cleaning the dataframe")
    print("Please wait...")
    # Drop unused columns
    df.drop(COLUMNS_TO_DROP,axis = 1,inplace=True)
    
    # Drop columns where percentage of NaN values is too high
    df = df.dropna(axis=1, thresh= len(df)*(1 - MAX_NAN_PERCENT_VALUE / 100), how='all') 
    
    # Drop rows with empty product_name or countries_en
    df = df[np.logical_and(
        np.logical_not(df['product_name'].isnull()),
        np.logical_not(df['countries_en'].isnull())
    )]
    
    # Drop duplicated rows in column product_name
    df.drop_duplicates(subset=['product_name'],inplace=True)

    print("Cleaning process terminated")
    
    return df

    

### Loading process

Now that our loading functions are defined, put some logic here to avoid long time processing while coding.

**Do not forget to set the global constant to Production values when coding is finished**

In [6]:
if (FORCE_LOAD_ORIGINAL_FILE == True) or path.exists(CLEANED_CSV_FILENAME) == False:
    df = loadOriginalTsvFile(ORIGINAL_TSV_FILENAME)
    df = cleanOriginalData(df)
    dumpCleanedCsvFile(df, CLEANED_CSV_FILENAME)
else:
    print("Cleaned CSV file found. Original data file processing is skipped")

    
if MAX_ENTRY_TO_LOAD != None:
    print("Loading the first",MAX_ENTRY_TO_LOAD,"rows from ",CLEANED_CSV_FILENAME)
else:
    print("Loading data from ",CLEANED_CSV_FILENAME)

print("Please wait...")
df = pd.read_csv(CLEANED_CSV_FILENAME,low_memory=False, nrows=MAX_ENTRY_TO_LOAD, index_col=0)
print("Dataframe loaded")

print('Number of rows   :',format(df.shape[0]))
print('Number of columns:',format(df.shape[1]))

Loading data from file data/OpenFoodFacts.tsv
Please wait...
Loading process terminated.
Cleaning the dataframe
Please wait...
Cleaning process terminated
Dumping the cleaned Dataframe into file data/OpenFoodFacts-cleaned.csv
Please wait...
Dumping process terminated
Loading the first 20000 rows from  data/OpenFoodFacts-cleaned.csv
Please wait...
Dataframe loaded
Number of rows   : 20000
Number of columns: 30


### Cleaning process

Now the we've loaded a *partially* cleaned dataframe, let's explore it and perform some more clean up.

First, we can ensure that we do not have any duplicated lines

In [7]:
print("Number of duplicated lines:",df.duplicated().sum())

Number of duplicated lines: 0


Ensure that we do not have duplicated values in the **product_name** columns and set it as Dataframe index

In [8]:
if 'product_name' in df.columns:
    if df['product_name'].duplicated().sum() == 0:
        print("There is no duplicated values in 'product_name' column. Set it as Dataframe index")
        df.set_index('product_name', inplace=True)
    else:
        print("WARNING: Duplicated values detected in 'product_name' column. Aborting Notebook.")
        exit(1)
else:
    print("Dataframe already indexed by 'product_name'")

There is no duplicated values in 'product_name' column. Set it as Dataframe index


Convert date and time columns into DateTime Pandas object, for smarter time manipulations.

During the first **to_datetime()** run, I've found values that was not convertible to a Datetime object. For example, one of the row contains value **Dia,Sogeres** in its **created_datetime** cell. Solution is to set the *unparsable* datetime strings to NaT using **error=coerce** paramter in **to_datetime()** call, and then fill NaT value with **fillna()** function using **ffill** method.

In [9]:
df['created_datetime'] = pd.to_datetime(df['created_datetime'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
df['created_datetime'].fillna(method = 'ffill', inplace = True)

df['last_modified_datetime'] = pd.to_datetime(df['last_modified_datetime'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
df['last_modified_datetime'].fillna(method = 'ffill', inplace = True)


In [10]:
df.describe()

,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,19438.000000,19438.000000,19438.000000,19484.000000,19387.000000,16613.000000,16013.000000,16066.000000,19397.000000,18158.000000,16161.000000,19412.000000,19341.000000,19341.000000,15398.000000,15613.000000,15553.000000,15786.000000,15958.000000,15958.000000
mean,2.342474,0.000875,0.035960,1138.608086,12.181831,4.968698,0.112226,0.019907,34.118356,16.866910,2.636399,6.989111,1.978487,0.778931,0.000155,0.010478,0.129003,0.003263,9.750533,9.719326
std,2.849387,0.029561,0.191105,784.983821,16.564434,7.560172,3.146284,0.038303,29.656419,21.553938,4.220313,8.492355,11.820749,4.653824,0.000766,0.310653,4.195635,0.119152,9.193572,9.204193
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.030000,0.000000,0.000000,-1.200000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,-10.000000
25%,0.000000,0.000000,0.000000,393.000000,0.000000,0.000000,0.000000,0.000000,6.670000,1.690000,0.000000,0.000000,0.076200,0.030000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,2.000000,0.000000,0.000000,1180.000000,4.760000,1.820000,0.000000,0.000000,24.620000,6.670000,1.500000,4.410000,0.683260,0.269000,0.000000,0.000000,0.033000,0.000950,11.000000,11.000000
75%,3.000000,0.000000,0.000000,1674.000000,20.000000,7.140000,0.000000,0.026000,61.900000,26.670000,3.600000,10.000000,1.541780,0.607000,0.000107,0.002800,0.101000,0.002250,17.000000,17.000000
max,29.000000,1.000000,3.000000,10757.000000,100.000000,92.860000,369.000000,0.900000,100.000000,100.000000,100.000000,100.000000,870.856780,342.857000,0.050700,35.714300,522.727000,12.121210,35.000000,35.000000
